In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap

In [ ]:
import ee
#ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import geemap
import ipyleaflet
import numpy as np
import requests
import os

In [ ]:
## specify areas of interest / districts and metadata
## URL method accessed an UrbanShift city's boundaries and uses information from file name and geoBoundaries properties ("shapeName", "shapeID") to create properties for output file
URL = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/ADM2/boundary-CRI-San_Jose-ADM2.geojson'
#'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/ADM1/boundary-MAR-Marrakech-ADM1.geojson'
#URL = 'https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/lab/tree/PlanetaryComputerExamples/urban_edge_t3.geojson'
DistrictsGJ = requests.get(URL).json()
Districts = geemap.geojson_to_ee(DistrictsGJ)
#Districts = ee.FeatureCollection('users/emackres/Wards/Addis_Ababa_Woredas')
DistrictsProjCRS = Districts.geometry().projection().crs()

print(DistrictsProjCRS.getInfo())
#print(Districts.limit(1).getInfo())

In [ ]:
# extract area properties from standarized filename
# https://note.nkmk.me/en/python-split-rsplit-splitlines-re/ 
basename = os.path.splitext(os.path.basename(URL))[0]
AOIname = basename.split('-',1)[1].rsplit('-',1)[0]

Areaofinterest = AOIname ## 3-letter country abreviation - city name with underscore for spaces, e.g. "ETH-Addis_Ababa"
print(Areaofinterest)

In [ ]:
## create map
Map = geemap.Map(height="350px")
Map

In [ ]:
## add basemap and center on area of interest
Map.add_basemap('HYBRID')
Map.centerObject(Districts, zoom=12)

In [ ]:
##Add Land use land cover dataset
WC = ee.ImageCollection("ESA/WorldCover/v100")
WorldCover = WC.first();

## define projection for use later
WCprojection = WC.first().projection();  
print('WorldCover projection:', WCprojection.getInfo());

Map.addLayer(WorldCover, {'bands': "Map"}, "WorldCover 10m 2020 (ESA)",1);

Map.add_legend(builtin_legend='ESA_WorldCover',position='bottomleft')

In [ ]:
## Add intra-urban land use dataset

ULU = ee.ImageCollection("projects/wri-datalab/urban_land_use/v1")

WRIulu = ULU.select('lulc').reduce(ee.Reducer.firstNonNull()).rename('lulc')
WRIulu = WRIulu.mask(WRIulu.mask().gt(0))
WRIroad = ULU.select('road_lulc').reduce(ee.Reducer.firstNonNull()).rename('lulc')
WRIuluwRoad = WRIulu.add(WRIroad).where(WRIroad.eq(1),6).mask(WRIulu.mask().gt(0))

ULUmaskedESA = WRIuluwRoad.updateMask(WorldCover.eq(50)) #.Or(WorldCover.eq(60)))

ULUmaskedESA = ULUmaskedESA.reproject(
      crs= WCprojection
    )

CLASSES_7=[
  "open_space",
  "nonresidential",
  "atomistic",
  "informal_subdivision",
  "formal_subdivision",
  "housing_project",
  "road"]
COLORS_7=[
  '33A02C',
  'E31A1C',
  'FB9A99',
  'FFFF99',
  '1F78B4',
  'A6CEE3',
  '3f3f3f']  
ULU7Params = {"bands": ['lulc'], 'min': 0, 'max': 6, "opacity": 1, "palette": COLORS_7}

Map.addLayer(ULUmaskedESA,ULU7Params,"Urban Land Use 2020 (WRI) masked to WorldCover built",True)

In [ ]:
## Calculate and add to map vegetation and water cover rasters

NDVIthreshold = 0.4 # decimal
NDWIthreshold = 0.3 # decimal


s2 = ee.ImageCollection("COPERNICUS/S2")

def addNDVI(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
  return image.addBands(ndvi)

def addNDWI(image):
  ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
  return image.addBands(ndwi)

withNDVI = s2.filterDate('2021-01-01', '2021-12-31').map(addNDVI).map(addNDWI);
##.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))


## Make a "greenest" pixel composite and masked version.
greenest = withNDVI.qualityMosaic('NDVI');
greenestNDVI = greenest.select('NDVI')
greenestMasked = greenestNDVI.updateMask(greenestNDVI.gte(NDVIthreshold))

## Make a "bluest" pixel composite and masked version.
bluest = withNDVI.qualityMosaic('NDWI');
bluestNDWI = bluest.select('NDWI')
bluestMasked = bluestNDWI.updateMask(bluestNDWI.gte(NDWIthreshold))

## Display the result.
visParams = {"bands": ['B4', 'B3', 'B2'], max: 0.3};
#Map.addLayer(greenest, visParams, 'Greenest pixel composite',0);
ndviParams = {"bands": ['NDVI'], min: -1, max: 1, "palette": ['blue', 'white', 'green']};
#Map.addLayer(greenestNDVI, ndviParams, 'Vegetation 2020: NDVI Greenest pixel composite',0);
Map.addLayer(greenestMasked, ndviParams, 'Vegetation 2020: NDVI composite masked at >'+str(NDVIthreshold)+' (Sentinel)');

ndwiParams = {"bands": ['NDWI'], min: -1, max: 1, "palette": ['green', 'white', 'blue']};
#Map.addLayer(bluestNDWI, ndwiParams, 'Water 2020: NDWI Bluest pixel composite',0);
Map.addLayer(bluestMasked, ndwiParams, 'Water 2020: NDWI composite masked at >'+str(NDWIthreshold)+' (Sentinel)');


In [ ]:
## add tree cover dataset
TML = ee.ImageCollection('projects/wri-datalab/TML')
TreeCover = TML.reduce(ee.Reducer.mean()).rename('b1')
TreePctThreshold = 10 #whole numbers - 0-100

## Reproject trees to match LULC projection


TreeCover = TreeCover.reproject(
      crs= WCprojection
    )

Map.addLayer(TreeCover.updateMask(TreeCover.gte(TreePctThreshold)),
             {'min':0, 'max':0.5, 'palette':['white','#006400']},
             'Tree Cover 2020 (WRI Trees in Mosaic Landscapes)',True,1)

In [ ]:
#add area of interest to map
#Map.addLayer(Districts,{},"Districts",True,0.5)


In [ ]:
## calculations to determine tree cover by LULC class


## function to create image of means of toCount for each asClass

def getmeanbyclass(classvalue):
    return ee.Image(toCount.updateMask(asClass.eq(classvalue)) #.And(toCount.gt(0))) # uncomment if you want includ only pixel that meet both criteria
                    # .unmask(0) # uncomment if you want to include all pixels not just pixels of classvalue
                    ).rename(ee.String('') #'class_count-'
                                       .cat(ee.Number(classvalue).toInt().format()))

## function to create image of count of each asClass
def getcountbyclass(classvalue):
    return ee.Image(toCount.updateMask(asClass.eq(classvalue)) #.And(toCount.gt(0))) # uncomment if you want includ only pixel that meet both criteria
                    # .unmask(0) # uncomment if you want to include all pixels not just pixels of classvalue
                    ).rename(ee.String('class_count-') #'class_count-'
                                      .cat(ee.Number(classvalue).toInt().format()))


In [ ]:
## create image with each WorldCover class mean as a band

asClass = WorldCover
toCount = TreeCover #greenestMasked #bluestMasked 

meanbyclass=ee.Image(getmeanbyclass(10)).addBands([
  getmeanbyclass(20),  
  getmeanbyclass(30),  
  getmeanbyclass(40),  
  getmeanbyclass(50),  
  getmeanbyclass(60),
  getmeanbyclass(70),  
    getmeanbyclass(80), 
    getmeanbyclass(90), 
    getmeanbyclass(95), 
    getmeanbyclass(100), 
])

## create image with each WorldCover class count as a band

countbyclass=ee.Image(getcountbyclass(10)).addBands([
  getcountbyclass(20),  
  getcountbyclass(30),  
  getcountbyclass(40),  
  getcountbyclass(50),  
  getcountbyclass(60),
  getcountbyclass(70),  
    getcountbyclass(80), 
    getcountbyclass(90), 
    getcountbyclass(95), 
    getcountbyclass(100), 
])

#print('meanbyclass', meanbyclass.getInfo())
#print('countbyclass', countbyclass.getInfo())
#Map.addLayer(meanbyclass.select('50'),{},"meanbyWCclass")

In [ ]:
## create image with each ULU class mean as a band

asClass = ULUmaskedESA
toCount = TreeCover #greenestMasked #bluestMasked 

meanbyclassULU=ee.Image(getmeanbyclass(0)).addBands([
  getmeanbyclass(1),  
  getmeanbyclass(2),  
  getmeanbyclass(3),  
  getmeanbyclass(4),  
  getmeanbyclass(5),
  getmeanbyclass(6) 
])

## create image with each ULU class count as a band

countbyclassULU=ee.Image(getcountbyclass(0)).addBands([
  getcountbyclass(1),  
  getcountbyclass(2),  
  getcountbyclass(3),  
  getcountbyclass(4),  
  getcountbyclass(5),
  getcountbyclass(6)
])
# Map.addLayer(meanbyclassULU.select('1'),{},"meanbyULUclass")

In [ ]:
## create FeatureCollection with mean of count for each class for each feature

histo=meanbyclass.reduceRegions(
  reducer= ee.Reducer.mean(), 
  collection= Districts, 
  scale= 10, 
  tileScale= 1
)

histo=countbyclass.reduceRegions(
  reducer= ee.Reducer.count(), 
  collection= histo, 
  scale= 10, 
  tileScale= 1
)

histo=meanbyclassULU.reduceRegions(
  reducer= ee.Reducer.mean(), 
  collection= histo, 
  scale= 10, 
  tileScale= 1
)

histo=countbyclassULU.reduceRegions(
  reducer= ee.Reducer.count(), 
  collection= histo, 
  scale= 10, 
  tileScale= 1
)

#print('histo:', histo.limit(1).getInfo())
print('histo:', histo.first().toDictionary().getInfo())

In [ ]:
## Define function to normalize count as percent of all pixels in each feature and create new properties with the values

def count_to_percent(feat):
    feat=ee.Feature(feat)
    hist=ee.Dictionary(feat.toDictionary(['10','20','30','40','50','60','70','80','90','95','100']))
    hist=hist.set('10',hist.get('10',0))
    hist=hist.set('20',hist.get('20',0))
    hist=hist.set('30',hist.get('30',0))
    hist=hist.set('40',hist.get('40',0))
    hist=hist.set('50',hist.get('50',0))
    hist=hist.set('60',hist.get('60',0))
    hist=hist.set('70',hist.get('70',0))
    hist=hist.set('80',hist.get('80',0))
    hist=hist.set('90',hist.get('90',0))
    hist=hist.set('95',hist.get('95',0))
    hist=hist.set('100',hist.get('100',0))
    
    def pct_hist(k,v):
        # convert whole number (0-100) to decimal percent (0-1)
        return ee.Number(v).multiply(ee.Number(0.01))
    
    pcts = hist.map(pct_hist)
    
    histC=ee.Dictionary(feat.toDictionary(['class_count-10','class_count-20','class_count-30','class_count-40','class_count-50','class_count-60','class_count-70','class_count-80','class_count-90','class_count-95','class_count-100']))
    histC=histC.set('10',histC.get('class_count-10',0))
    histC=histC.set('20',histC.get('class_count-20',0))
    histC=histC.set('30',histC.get('class_count-30',0))
    histC=histC.set('40',histC.get('class_count-40',0))
    histC=histC.set('50',histC.get('class_count-50',0))
    histC=histC.set('60',histC.get('class_count-60',0))
    histC=histC.set('70',histC.get('class_count-70',0))
    histC=histC.set('80',histC.get('class_count-80',0))
    histC=histC.set('90',histC.get('class_count-90',0))
    histC=histC.set('95',histC.get('class_count-95',0))
    histC=histC.set('100',histC.get('class_count-100',0))
    
    def area_hist(k,v):
        # convert 10m pixel count of class to KM2 of class
        return ee.Number(v).multiply(ee.Number(100)).multiply(ee.Number(0.000001))
    
    classAreas = histC.map(area_hist)
    
    FeatArea = feat.area(0.001).multiply(0.000001)
    cityID = Areaofinterest
    geo_level = feat.getString("shapeID").split('-').getString(1)
    #geo_name = feat.getString("Sub_City").cat(ee.String("-")).cat(feat.getString("Woreda"))
    #geo_name = feat.getString("city_name_viz").split(' ').join('_')
    geo_name = feat.getString("shapeName").split(' ').join('_')
    geo_id = ee.String(cityID+"-").cat(geo_name)
    year = 2020
    source = "ESA WorldCover, WRI TML, WRI ULU"

    totalPixels=hist.values()
    
    histULU=ee.Dictionary(feat.toDictionary(['0','1','2','3','4','5','6']))
    histULU=histULU.set('0',histULU.get('0',0))
    histULU=histULU.set('1',histULU.get('1',0))
    histULU=histULU.set('2',histULU.get('2',0))
    histULU=histULU.set('3',histULU.get('3',0))
    histULU=histULU.set('4',histULU.get('4',0))
    histULU=histULU.set('5',histULU.get('5',0))
    histULU=histULU.set('6',histULU.get('6',0))
    
    pctsULU = histULU.map(pct_hist)
        
    histULUc=ee.Dictionary(feat.toDictionary(['class_count-0','class_count-1','class_count-2','class_count-3','class_count-4','class_count-5','class_count-6']))
    histULUc=histULUc.set('0',histULUc.get('class_count-0',0))
    histULUc=histULUc.set('1',histULUc.get('class_count-1',0))
    histULUc=histULUc.set('2',histULUc.get('class_count-2',0))
    histULUc=histULUc.set('3',histULUc.get('class_count-3',0))
    histULUc=histULUc.set('4',histULUc.get('class_count-4',0))
    histULUc=histULUc.set('5',histULUc.get('class_count-5',0))
    histULUc=histULUc.set('6',histULUc.get('class_count-6',0))

    classAreasULU = histULUc.map(area_hist)

    return feat.set({
        'LC10treepct': pcts.getNumber('10'),
        'LC20treepct': pcts.getNumber('20'),
        'LC30treepct': pcts.getNumber('30'),
        'LC40treepct': pcts.getNumber('40'),
        'LC50treepct': pcts.getNumber('50'),
        'LC60treepct': pcts.getNumber('60'),
        'LC70treepct': pcts.getNumber('70'),
        'LC80treepct': pcts.getNumber('80'),
        'LC90treepct': pcts.getNumber('90'),
        'LC95treepct': pcts.getNumber('95'),
        'LC100treepct': pcts.getNumber('100'),
        'TotalareaKM2': FeatArea,
        'TotalPixels': totalPixels,
        'geo_level': geo_level,
        'geo_name': geo_name,
        'geo_id': geo_id,
        'year':year,
        'source':source,
        'LC10areaKM2': classAreas.getNumber('10'),
        'LC20areaKM2': classAreas.getNumber('20'),
        'LC30areaKM2': classAreas.getNumber('30'),
        'LC40areaKM2': classAreas.getNumber('40'),
        'LC50areaKM2': classAreas.getNumber('50'),
        'LC60areaKM2': classAreas.getNumber('60'),
        'LC70areaKM2': classAreas.getNumber('70'),
        'LC80areaKM2': classAreas.getNumber('80'),
        'LC90areaKM2': classAreas.getNumber('90'),
        'LC95areaKM2': classAreas.getNumber('95'),
        'LC100areaKM2': classAreas.getNumber('100'),
        'ULU0treepct': pctsULU.getNumber('0'),
        'ULU1treepct': pctsULU.getNumber('1'),
        'ULU2treepct': pctsULU.getNumber('2'),
        'ULU3treepct': pctsULU.getNumber('3'),
        'ULU4treepct': pctsULU.getNumber('4'),
        'ULU5treepct': pctsULU.getNumber('5'),
        'ULU6treepct': pctsULU.getNumber('6'),
        'ULU0areaKM2': classAreasULU.getNumber('0'),
        'ULU1areaKM2': classAreasULU.getNumber('1'),
        'ULU2areaKM2': classAreasULU.getNumber('2'),
        'ULU3areaKM2': classAreasULU.getNumber('3'),
        'ULU4areaKM2': classAreasULU.getNumber('4'),
        'ULU5areaKM2': classAreasULU.getNumber('5'),
        'ULU6areaKM2': classAreasULU.getNumber('6'),
    })

In [ ]:
## update FeatureCollection with percents

tree_pcts=histo.map(count_to_percent)

#print('Tree cover % by Land Cover class for Districts',tree_pcts.limit(1).getInfo());
print('Tree cover % by Land Cover class for Districts',tree_pcts.first().toDictionary().getInfo())

In [ ]:
## render on map percent tree cover by class from feature collection

empty = ee.Image().byte()
Tpctfills = empty.paint(**{'featureCollection': tree_pcts,'color': 'ULU1treepct'})

fillspalette = ['red', 'green']
Map.addLayer(Tpctfills, {'palette': fillspalette,'min':0,'max':0.15}, '% of Non-res built area with tree cover', True, 0.65)
Map

In [ ]:
## select properties to keep, sort features and create data frame to display properties
tree_pctsSort = tree_pcts.select([
    'TotalareaKM2',
    'geo_level',
    'geo_name',
    'geo_id',
    'year',
    'source',
    'LC10areaKM2','LC20areaKM2','LC30areaKM2','LC40areaKM2','LC50areaKM2','LC60areaKM2','LC70areaKM2','LC80areaKM2','LC90areaKM2','LC95areaKM2','LC100areaKM2',
    'LC10treepct','LC20treepct','LC30treepct','LC40treepct','LC50treepct','LC60treepct','LC70treepct','LC80treepct','LC90treepct','LC95treepct','LC100treepct',
    'ULU0treepct','ULU1treepct','ULU2treepct','ULU3treepct','ULU4treepct','ULU5treepct','ULU6treepct',
    'ULU0areaKM2','ULU1areaKM2','ULU2areaKM2','ULU3areaKM2','ULU4areaKM2','ULU5areaKM2','ULU6areaKM2'
]).sort('LC50treepct', False) #
#print('Tree cover sorted version',tree_pctsSort.limit(1).getInfo());

In [ ]:
df = geemap.ee_to_pandas(tree_pctsSort)
df

In [ ]:
## display features in chart

import geemap.chart as chart

xProperty = 'geo_name' #,"Woreda"
yProperties = ['LC50treepct'] # ,'LC50areaKM2'

options = {
    'xlabel': "District",
    'ylabel': "% tree cover in built areas",
    "legend_location": "top-right",
    "height": "500px",
}

chart.feature_byFeature(tree_pctsSort, xProperty, yProperties, **options)